In [1]:
import requests #gets the HTML 
import lxml.html as lh #parsing 
import pandas as pd #datasets
from bs4 import BeautifulSoup # html parsing
from requests_html import AsyncHTMLSession # async session
from requests_html import HTMLSession # sync session
import numpy as np

We can access the content of a web page trough the requests API's get() method. We desire to parse the HTML to retrieve the table containing our data. However, the data in inside a table which gets rendered by the browser, so we can't simply dig into the HTML. Let's start by installing the requests_html package. To install, simply write pip install requests_html in your shell. I'll use AsyncHTMLSession since I'm working with Jupyter, but you can use HTMLSession aswell. After retrieving the page, we render the javascript code to obtain the complete HTML code. I'll post both the asynchornous function (the one I'm using currently) and the standard function (if you might need it). 



To do this, we use the BeautifulSoulp4 library. After generating a bs4 object, you can either inspect the html trough your browser or using the prettify() method of bs4. 

In [2]:
# Retrieve a beautifoulsoup object with the HTML of the url (asynchronous)
async def asyncHTMLfromURL(url):
    # create an HTML asynchornous session
    asession = AsyncHTMLSession()
    #use the session object to connect to the page
    page = await asession.get(url)
    # return the HTTP response code
    print('HTTP response: ' + str(page.status_code))    # 200: OK; 204: NO CONTENT
    # Run JavaScript code on webpage (sleep is for the loading time of the contents)
    await page.html.arender(sleep=5)

    # create a bs4 object over the html of the page
    soup = BeautifulSoup(page.html.html, 'html.parser')
    return soup

In [3]:
# Retrieve a beautifoulsoup object with the HTML of the url)
def HTMLfromURL(url):
    # create an HTML asynchornous session
    session = HTMLSession()
    #use the session object to connect to the page
    page = session.get(url)
    # return the HTTP response code
    print('HTTP response: ' + str(page.status_code))   # 200: OK; 204: NO CONTENT
    # Run JavaScript code on webpage (sleep is for the loading time of the contents)
    page.html.render(sleep=4.5)

    # create a bs4 object over the html of the page
    soup = BeautifulSoup(page.html.html, 'html.parser')
    return soup

We locate our target data over the table with class 'days ng-star-inserted'.  after retrieving the table, we can easily access the headings and the body of our table. We can notice that there's an inner table, so we locate the html and estract the data from the various rows. We also procees to split the multi-rows which contains 'Max', 'Avg' and 'Min' of various features, and we reallocate them as new features. I preferred to do this manually because extracting from the <td> ws a bit tricky for this table.

In [4]:
def dataRetrieval(soup):
    #retrieve a bs4 object containing our table 
    weather_table = soup.find('table', class_='days ng-star-inserted')
    #retrieve the headings of the table
    weather_table_head = weather_table.thead.find_all("tr")

    # Get all the headings
    headings = []
    for td in weather_table_head[0].find_all("td"):
        headings.append(td.text)
    #print(headings)


    ## retrieve the data from the table
    column, data = [], []
    weather_table_data = weather_table.tbody.find_all('table') # find inner tables
    for table in weather_table_data:
        weather_table_rows = table.find_all('tr') # find rows
        for row in weather_table_rows:  
            column.append(row.text.strip()) #take data without spaces 
        data.append(column)
        column = []
    # slice the triple rows into sub-wors
    datas = slicing(data, column, headings)
    return datas

In [5]:
## split triples into single features
def slicing(data, column, headings):
     
    datas, col, colmax, colavg, colminy = [], [],[],[], []
    split, first, i = False, True, 0
    
    for column in data:
        for entry in column:
            for c in entry:
                if c == ' ':
                    split = True
            if split:
                maxy, avg, miny = entry.split()
                if first:
                    colmax.append(headings[i]+ ' '+ maxy)
                    colavg.append(headings[i]+ ' '+ avg)
                    colminy.append(headings[i]+ ' '+ miny)
                    first = False
                else:
                    colmax.append(maxy)
                    colavg.append(avg)
                    colminy.append(miny)
            elif first:
                col.append(headings[i]+ ' '+entry)
                first = False
            else: col.append(entry)
        if split:
            datas.append(colmax)
            datas.append(colavg)
            datas.append(colminy)
            colmax, colavg, colminy = [],[],[]
            split = False
        else: 
            datas.append(col)
            col = []
        first = True
        i +=1 
    return datas

We can reformat the data to be more similar to the water level dataset and also more compact, so it can store both day, month and year. 

In [6]:
## reformat the data section to be similar to the water level datas (days/month/year)
def dataReformat(data, year, month):
    data[0][0] = 'Dates'
    for i in range(1, len(data[0])):
        if len(data[0][i]) == 2:
            if (len(str(month)) == 2):
                data[0][i] = data[0][i] + '/' + str(month) + '/' + str(year)
            else: data[0][i] = data[0][i] + '/' + '0' + str(month) + '/' + str(year)
        elif (len(str(month)) == 2):
                data[0][i] = '0' + data[0][i] + '/' + str(month) + '/' + str(year)
        else: data[0][i] = '0' + data[0][i] + '/' + '0' + str(month) + '/' + str(year)

We can now retrieve as much data as possible. The more the better. We start from 1997 (the first weather data available on the site for venice) and go up to 2018 (this is due to the water level data which goes up to 2018). We procees to store all of our data in a pandas DataFrame. This might take a while so be aware of it.  

In [ ]:
## double for - retrieve the various year's data
first = True
for year in range (1997, 2019):
    for month in range (1, 13):
        print('Retrieving weather data of '+str(month)+'/'+str(year))
        # URL of the page we want to retrieve
        url='https://www.wunderground.com/history/monthly/it/venice/date/'+str(year)+'-'+str(month)
        # Retrieve the HTML form the url
        soup = await asyncHTMLfromURL(url)
        # retrieve the data from the HTML
        fulldata = dataRetrieval(soup)
        # reformat the data section to be similar to the water level datas
        dataReformat(fulldata, year, month)
        dataframe = np.array([item[1:] for item in fulldata]).transpose()
        if first:
            df = pd.DataFrame(dataframe, columns=([item[0] for item in fulldata]))
            first = False
        else:
            df2 = pd.DataFrame(dataframe, columns=([item[0] for item in fulldata]))
            df = df.append(df2, ignore_index=True)
        print('weather data of '+str(month)+'/'+str(year)+' retrieved successfully!')

Now we want to save our dataframe so that we don't actually have to download it every single time. I'm going to use a .CSV format which is pretty standard way to save a dataset. This can be done easily trough the 'to_csv()' function from pandas.DataFrame

In [11]:
#save dataframe as .csv
df.to_csv(r'.\weather_data.csv')

Now that we've prepared our hystorical weather data of Venice, we want to obtain and prepare a clean dataset of the sea level over the last few years. Luckily, we can find the data we need in the following page https://www.comune.venezia.it/it/content/archivio-storico-livello-marea-venezia-1 .
We need however to download every .csv and rearrange them in a single dataset, which we'll later join with the other dataset we've generated previously. We could do this manually, but let's use some python magic to automate the procedure! Once again to move around the various urls we can jsut change the year from the url, and trough the request.get() function we can download every .csv which we'll then write on the disk.

In [ ]:
import requests
import pandas as pd
import numpy as np
from statistics import mean, median

## from every url download and save on disk the .csv 
for year in range (1997, 2019):
    url = 'https://www.comune.venezia.it/sites/comune.venezia.it/files/documenti/centro_maree/archivioDati/valoriorari_puntasalute_'+str(year)+'.csv'
    obj = requests.get(url)

    with open('./sea_level/'+str(year)+'_sea_levels.csv', 'wb') as file:
        file.write(obj.content)

Much quicker than do it manually right? 
By checking our .csv, we discover that the data are arranged hourly for each day, while we only have one single weather value for every day. To play around this, we can extract the maximum, the minium and the average of the sea levels for each day. (Note that the 2015 files has some rows that contains plain text at the end that needs to be removed!)
To load the csv we can use Pandas. Since the file use the ';' as separator, we need to specify it (since pandas usually have ',' as default).

In [ ]:
data_sea = pd.read_csv('./sea_level/'+str(2018)+'_sea_levels.csv', sep=";")

The idea now is to cycle between every year, extract the info we need for every set of days and re-arrange everything in a new dataset. The data column name change from 2016 onwards from 'Data' to 'data' and 'GIORNO', so keep that in mind while extracting the data. Using functions will make the code cleaner, so don't just paste the same code over and over! Moreover, the year 2018 has metres as a mestric, so we'll adapt it to cm as the rest of the data.

In [ ]:

## create a new dataset using the unorganized data 
def sea_refactor(data_sea, day, first, df):
    sea_level = []
    for index, row in data_sea.iterrows():
        # appends all the multiple data for single day
        if (str(row['Date']) == day):
            sea_level.append(row['Sea_level'])
        else:
            # only first occurance: create a new dataset containing rearranged data 
            if first:
                df = pd.DataFrame(np.array([[day, min(sea_level), "{0:.2f}".format(mean(sea_level)), max(sea_level)]]), columns=['day', 'min_sea_level (cm)', 'avg_sea_level (cm)', 'max_sea_level (cm)'])
                first = False
            # append to the previous dataset the new row with rearranged data   
            else:
                df2 = pd.DataFrame(np.array([[day, min(sea_level), "{0:.2f}".format(mean(sea_level)), max(sea_level)]]), columns=(['day', 'min_sea_level (cm)', 'avg_sea_level (cm)', 'max_sea_level (cm)']))
                df = df.append(df2, ignore_index=True)
            # update indexes
            day = str(row['Date'])
            sea_level = []
            sea_level.append(row['Sea_level']) 
    return df

## change the metric from m to cm 
def sea_leveller(dataset):                             
    for index, row in dataset.iterrows():
        dataset.at[index, 'Sea_level'] = (row['Sea_level']*100)
    return dataset

## change the format of the year over the date for the whole dataset
def year_refactor(dataset):
    for index, row in dataset.iterrows():
        day, month, _ = row['Date'].split('/')
        dataset.at[index, 'Date'] = (str(day)+'/'+str(month)+'/'+str(2007))

# change the format of the date for the whole dataset
def date_reformer(dataset, year):
    for index, row in dataset.iterrows():
        day, month, _ = row['Date'].split('-')
        if len(str(day)) == 1: day = '0'+str(day)
        if str(month) == 'gen': dataset.at[index, 'Date'] = (str(day)+'/0'+str(1)+'/'+str(year))
        elif str(month) == 'feb': dataset.at[index, 'Date'] = (str(day)+'/0'+str(2)+'/'+str(year))
        elif str(month) == 'mar': dataset.at[index, 'Date'] = (str(day)+'/0'+str(3)+'/'+str(year))
        elif str(month) == 'apr': dataset.at[index, 'Date'] = (str(day)+'/0'+str(4)+'/'+str(year))
        elif str(month) == 'mag': dataset.at[index, 'Date'] = (str(day)+'/0'+str(5)+'/'+str(year))
        elif str(month) == 'giu': dataset.at[index, 'Date'] = (str(day)+'/0'+str(6)+'/'+str(year))
        elif str(month) == 'lug': dataset.at[index, 'Date'] = (str(day)+'/0'+str(7)+'/'+str(year))
        elif str(month) == 'ago': dataset.at[index, 'Date'] = (str(day)+'/0'+str(8)+'/'+str(year))
        elif str(month) == 'set': dataset.at[index, 'Date'] = (str(day)+'/0'+str(9)+'/'+str(year))
        elif str(month) == 'ott': dataset.at[index, 'Date'] = (str(day)+'/'+str(10)+'/'+str(year))
        elif str(month) == 'nov': dataset.at[index, 'Date'] = (str(day)+'/'+str(11)+'/'+str(year))
        elif str(month) == 'dic': dataset.at[index, 'Date'] = (str(day)+'/'+str(12)+'/'+str(year))
    return dataset

## generate a new dataset from previous disorganized data using previosuly defined functions
first, df = True, []
for year in range(1997, 2019):
    # load the dataset
    data_sea = pd.read_csv('./sea_level/'+str(year)+'_sea_levels.csv', sep=";")
    day = '01/01/'+str(year)
    # rename the columns of the dataset 
    data_sea = data_sea.rename(columns={str(data_sea.columns[0]):'Date', str(data_sea.columns[2]):'Sea_level'})

    if year == 2018: data_sea = sea_leveller(data_sea)
        
    if year == 2007: year_refactor(data_sea)
        
    if year in [x for x in range(1997, 2010) if x != 2007]: 
        data_sea = date_reformer(data_sea, year)
        
    print('Processing the year '+str(year))
    df = sea_refactor(data_sea, day, first, df)
    print('Done processing the year '+str(year))
    first = False      

And finally we can save of dataframe as a .csv which we'll load later on.

In [ ]:
#save dataframe as .csv
df.to_csv(r'.\sea_level.csv')